In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules.builder import *
from modules.cleaner import *
from modules.builder import Builder
import pandas as pd
import os
from modules.cleaner import Cleaner
from modules.qualityChecker import QualityChecker
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import joblib
import plotly.express as px
from sklearn.metrics import r2_score
import opendatasets as od

In [3]:
od.download('https://www.kaggle.com/datasets/caesarmario/our-world-in-data-covid19-dataset')
od.download('https://www.kaggle.com/datasets/rsrishav/covid-vaccination-dataset')

Skipping, found downloaded files in "./our-world-in-data-covid19-dataset" (use force=True to force download)
Skipping, found downloaded files in "./covid-vaccination-dataset" (use force=True to force download)


In [4]:
df = pd.read_csv("/usr/src/covid_prediction/our-world-in-data-covid19-dataset/owid-covid-data.csv")

In [5]:
pd.set_option('display.max_rows', 500)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206397 entries, 0 to 206396
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    206397 non-null  object 
 1   continent                                   194465 non-null  object 
 2   location                                    206397 non-null  object 
 3   date                                        206397 non-null  object 
 4   total_cases                                 198013 non-null  float64
 5   new_cases                                   197775 non-null  float64
 6   new_cases_smoothed                          196598 non-null  float64
 7   total_deaths                                179247 non-null  float64
 8   new_deaths                                  179213 non-null  float64
 9   new_deaths_smoothed                         178046 non-null  float64
 

In [7]:
df.loc[df['location'] == 'Thailand']

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
183133,THA,Asia,Thailand,2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183134,THA,Asia,Thailand,2020-01-05,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183135,THA,Asia,Thailand,2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183136,THA,Asia,Thailand,2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183137,THA,Asia,Thailand,2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184072,THA,Asia,Thailand,2022-07-31,4592284.0,2108.0,2098.714,31404.0,19.0,30.714,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
184073,THA,Asia,Thailand,2022-08-01,4594127.0,1843.0,2362.000,31431.0,27.0,34.571,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
184074,THA,Asia,Thailand,2022-08-02,4596559.0,2432.0,2055.857,31463.0,32.0,29.286,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
184075,THA,Asia,Thailand,2022-08-03,4598725.0,2166.0,2093.571,31492.0,29.0,28.857,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN


In [8]:
df = df.loc[df['location'] == 'Thailand']

In [9]:
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
183133,THA,Asia,Thailand,2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183134,THA,Asia,Thailand,2020-01-05,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183135,THA,Asia,Thailand,2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183136,THA,Asia,Thailand,2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183137,THA,Asia,Thailand,2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184072,THA,Asia,Thailand,2022-07-31,4592284.0,2108.0,2098.714,31404.0,19.0,30.714,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
184073,THA,Asia,Thailand,2022-08-01,4594127.0,1843.0,2362.000,31431.0,27.0,34.571,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
184074,THA,Asia,Thailand,2022-08-02,4596559.0,2432.0,2055.857,31463.0,32.0,29.286,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
184075,THA,Asia,Thailand,2022-08-03,4598725.0,2166.0,2093.571,31492.0,29.0,28.857,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN


In [10]:
df.isna().sum()

iso_code                                        0
continent                                       0
location                                        0
date                                            0
total_cases                                    18
new_cases                                      21
new_cases_smoothed                             26
total_deaths                                   57
new_deaths                                     58
new_deaths_smoothed                            63
total_cases_per_million                        18
new_cases_per_million                          21
new_cases_smoothed_per_million                 26
total_deaths_per_million                       57
new_deaths_per_million                         58
new_deaths_smoothed_per_million                63
reproduction_rate                              74
icu_patients                                  944
icu_patients_per_million                      944
hosp_patients                                 944


In [11]:
df.loc[df['total_cases'].isnull()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
183133,THA,Asia,Thailand,2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183134,THA,Asia,Thailand,2020-01-05,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183135,THA,Asia,Thailand,2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183136,THA,Asia,Thailand,2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183137,THA,Asia,Thailand,2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183138,THA,Asia,Thailand,2020-01-09,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183139,THA,Asia,Thailand,2020-01-10,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183140,THA,Asia,Thailand,2020-01-11,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183141,THA,Asia,Thailand,2020-01-12,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN
183142,THA,Asia,Thailand,2020-01-13,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9,38.8,90.67,2.1,77.15,0.777,NaN,NaN,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 944 entries, 183133 to 184076
Data columns (total 67 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   iso_code                                    944 non-null    object 
 1   continent                                   944 non-null    object 
 2   location                                    944 non-null    object 
 3   date                                        944 non-null    object 
 4   total_cases                                 926 non-null    float64
 5   new_cases                                   923 non-null    float64
 6   new_cases_smoothed                          918 non-null    float64
 7   total_deaths                                887 non-null    float64
 8   new_deaths                                  886 non-null    float64
 9   new_deaths_smoothed                         881 non-null    float64
 10  total_

In [13]:
df['tests_units'].unique()

array(['tests performed', nan], dtype=object)

In [14]:
df['tests_units'] = np.where(df['tests_units'] =='tests performed', 1, 0)

In [15]:
df['tests_units']

183133    1
183134    1
183135    1
183136    1
183137    1
         ..
184072    0
184073    0
184074    0
184075    0
184076    0
Name: tests_units, Length: 944, dtype: int64

In [16]:
df.isna().sum()

iso_code                                        0
continent                                       0
location                                        0
date                                            0
total_cases                                    18
new_cases                                      21
new_cases_smoothed                             26
total_deaths                                   57
new_deaths                                     58
new_deaths_smoothed                            63
total_cases_per_million                        18
new_cases_per_million                          21
new_cases_smoothed_per_million                 26
total_deaths_per_million                       57
new_deaths_per_million                         58
new_deaths_smoothed_per_million                63
reproduction_rate                              74
icu_patients                                  944
icu_patients_per_million                      944
hosp_patients                                 944


In [17]:
df['tests_per_case'].loc[df['tests_per_case'].notnull()]

183157    27.3
183158    37.8
183159    48.3
183160    43.5
183161    61.9
          ... 
184011     3.2
184012     3.3
184013     3.3
184014     3.2
184015     3.2
Name: tests_per_case, Length: 857, dtype: float64

In [18]:
columns = [
    'icu_patients_per_million',
"hosp_patients",
"hosp_patients_per_million",
"weekly_icu_admissions",
'weekly_icu_admissions_per_million',
"weekly_hosp_admissions",
'weekly_hosp_admissions_per_million',
    'excess_mortality_cumulative_absolute', 'excess_mortality_cumulative', 'excess_mortality', 'excess_mortality_cumulative_per_million'
]
df = df.drop(columns=columns)

In [19]:
df = df.reset_index()

In [20]:
df.isna().sum()

index                                           0
iso_code                                        0
continent                                       0
location                                        0
date                                            0
total_cases                                    18
new_cases                                      21
new_cases_smoothed                             26
total_deaths                                   57
new_deaths                                     58
new_deaths_smoothed                            63
total_cases_per_million                        18
new_cases_per_million                          21
new_cases_smoothed_per_million                 26
total_deaths_per_million                       57
new_deaths_per_million                         58
new_deaths_smoothed_per_million                63
reproduction_rate                              74
icu_patients                                  944
total_tests                                    62


In [21]:
df['date'].loc[df['total_vaccinations'].isnull()]

0      2020-01-04
1      2020-01-05
2      2020-01-06
3      2020-01-07
4      2020-01-08
5      2020-01-09
6      2020-01-10
7      2020-01-11
8      2020-01-12
9      2020-01-13
10     2020-01-14
11     2020-01-15
12     2020-01-16
13     2020-01-17
14     2020-01-18
15     2020-01-19
16     2020-01-20
17     2020-01-21
18     2020-01-22
19     2020-01-23
20     2020-01-24
21     2020-01-25
22     2020-01-26
23     2020-01-27
24     2020-01-28
25     2020-01-29
26     2020-01-30
27     2020-01-31
28     2020-02-01
29     2020-02-02
30     2020-02-03
31     2020-02-04
32     2020-02-05
33     2020-02-06
34     2020-02-07
35     2020-02-08
36     2020-02-09
37     2020-02-10
38     2020-02-11
39     2020-02-12
40     2020-02-13
41     2020-02-14
42     2020-02-15
43     2020-02-16
44     2020-02-17
45     2020-02-18
46     2020-02-19
47     2020-02-20
48     2020-02-21
49     2020-02-22
50     2020-02-23
51     2020-02-24
52     2020-02-25
53     2020-02-26
54     2020-02-27
55     202

In [22]:
df.at[515, 'total_vaccinations'] = df.at[514, 'total_vaccinations'] + (df.at[516, 'total_vaccinations'] - df.at[514, 'total_vaccinations'])

In [23]:
df.at[515, 'total_vaccinations']

3961589.0

In [24]:
for i in range(410):
    df.at[i, 'total_vaccinations'] = 0

In [25]:
df['date'].loc[df['total_vaccinations'].isnull()]

410    2021-02-17
411    2021-02-18
412    2021-02-19
413    2021-02-20
414    2021-02-21
415    2021-02-22
416    2021-02-23
417    2021-02-24
418    2021-02-25
419    2021-02-26
421    2021-02-28
424    2021-03-03
425    2021-03-04
426    2021-03-05
427    2021-03-06
428    2021-03-07
429    2021-03-08
430    2021-03-09
432    2021-03-11
433    2021-03-12
435    2021-03-14
436    2021-03-15
437    2021-03-16
438    2021-03-17
439    2021-03-18
440    2021-03-19
441    2021-03-20
442    2021-03-21
443    2021-03-22
444    2021-03-23
446    2021-03-25
447    2021-03-26
448    2021-03-27
449    2021-03-28
450    2021-03-29
452    2021-03-31
466    2021-04-14
496    2021-05-14
497    2021-05-15
588    2021-08-14
605    2021-08-31
628    2021-09-23
629    2021-09-24
630    2021-09-25
631    2021-09-26
632    2021-09-27
633    2021-09-28
634    2021-09-29
635    2021-09-30
636    2021-10-01
637    2021-10-02
638    2021-10-03
639    2021-10-04
685    2021-11-19
696    2021-11-30
943    202

In [26]:
df_vac = pd.read_csv('/usr/src/covid_prediction/covid-vaccination-dataset/vaccinations.csv')

In [27]:
df_vac = df_vac.loc[df_vac['location'] == 'Thailand']

In [28]:
df_vac['date'] = pd.to_datetime(df_vac['date'], format='%Y-%m-%d')

In [29]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [30]:
#df_vac['total_vaccinations'] = df_vac.apply(lambda x: x['daily_vaccinations'] if x['total_vaccinations'] == float("nan") else x['daily_vaccinations'], axis=1)

In [31]:
df.set_index('date', inplace=True)
df.update(df_vac.set_index('date'))
df = df.reset_index()

In [32]:
df_vac.loc[df_vac['date'] == '2021-11-30']

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred
105098,Thailand,THA,2021-11-30,NaN,NaN,NaN,NaN,NaN,590665.0,NaN,NaN,NaN,NaN,8249.0,223601.0,0.312


In [33]:
df.at[943, 'total_vaccinations'] = 141814894

In [34]:
df_vac.loc[df_vac['date'] == '2021-11-30']

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred
105098,Thailand,THA,2021-11-30,NaN,NaN,NaN,NaN,NaN,590665.0,NaN,NaN,NaN,NaN,8249.0,223601.0,0.312


In [35]:
df['date'].loc[df['total_vaccinations'].isnull()]

410   2021-02-17
411   2021-02-18
412   2021-02-19
413   2021-02-20
414   2021-02-21
415   2021-02-22
416   2021-02-23
417   2021-02-24
418   2021-02-25
419   2021-02-26
421   2021-02-28
424   2021-03-03
425   2021-03-04
426   2021-03-05
427   2021-03-06
428   2021-03-07
429   2021-03-08
430   2021-03-09
432   2021-03-11
433   2021-03-12
435   2021-03-14
436   2021-03-15
437   2021-03-16
438   2021-03-17
439   2021-03-18
440   2021-03-19
441   2021-03-20
442   2021-03-21
443   2021-03-22
444   2021-03-23
446   2021-03-25
447   2021-03-26
448   2021-03-27
449   2021-03-28
450   2021-03-29
452   2021-03-31
466   2021-04-14
496   2021-05-14
497   2021-05-15
588   2021-08-14
605   2021-08-31
628   2021-09-23
629   2021-09-24
630   2021-09-25
631   2021-09-26
632   2021-09-27
633   2021-09-28
634   2021-09-29
635   2021-09-30
636   2021-10-01
637   2021-10-02
638   2021-10-03
639   2021-10-04
685   2021-11-19
696   2021-11-30
Name: date, dtype: datetime64[ns]

In [36]:
df['total_vaccinations']

0              0.0
1              0.0
2              0.0
3              0.0
4              0.0
          ...     
939    140829069.0
940    140859084.0
941    140893847.0
942    140926438.0
943    141814894.0
Name: total_vaccinations, Length: 944, dtype: float64

In [37]:
df['total_vaccinations'].loc[df['date'] == '2021-02-17']

410   NaN
Name: total_vaccinations, dtype: float64

In [38]:
df = df.fillna(0)

In [39]:
df

,date,index,iso_code,continent,location,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,2020-01-04,183133,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,16277.671,0.1,109.861,7.04,1.9,38.8,90.67,2.1,77.15,0.777
1,2020-01-05,183134,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,16277.671,0.1,109.861,7.04,1.9,38.8,90.67,2.1,77.15,0.777
2,2020-01-06,183135,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,16277.671,0.1,109.861,7.04,1.9,38.8,90.67,2.1,77.15,0.777
3,2020-01-07,183136,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,16277.671,0.1,109.861,7.04,1.9,38.8,90.67,2.1,77.15,0.777
4,2020-01-08,183137,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,16277.671,0.1,109.861,7.04,1.9,38.8,90.67,2.1,77.15,0.777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,2022-07-31,184072,THA,Asia,Thailand,4592284.0,2108.0,2098.714,31404.0,19.0,...,16277.671,0.1,109.861,7.04,1.9,38.8,90.67,2.1,77.15,0.777
940,2022-08-01,184073,THA,Asia,Thailand,4594127.0,1843.0,2362.000,31431.0,27.0,...,16277.671,0.1,109.861,7.04,1.9,38.8,90.67,2.1,77.15,0.777
941,2022-08-02,184074,THA,Asia,Thailand,4596559.0,2432.0,2055.857,31463.0,32.0,...,16277.671,0.1,109.861,7.04,1.9,38.8,90.67,2.1,77.15,0.777
942,2022-08-03,184075,THA,Asia,Thailand,4598725.0,2166.0,2093.571,31492.0,29.0,...,16277.671,0.1,109.861,7.04,1.9,38.8,90.67,2.1,77.15,0.777


In [40]:
df.isna().sum()

date                                          0
index                                         0
iso_code                                      0
continent                                     0
location                                      0
total_cases                                   0
new_cases                                     0
new_cases_smoothed                            0
total_deaths                                  0
new_deaths                                    0
new_deaths_smoothed                           0
total_cases_per_million                       0
new_cases_per_million                         0
new_cases_smoothed_per_million                0
total_deaths_per_million                      0
new_deaths_per_million                        0
new_deaths_smoothed_per_million               0
reproduction_rate                             0
icu_patients                                  0
total_tests                                   0
new_tests                               

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 57 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   date                                        944 non-null    datetime64[ns]
 1   index                                       944 non-null    int64         
 2   iso_code                                    944 non-null    object        
 3   continent                                   944 non-null    object        
 4   location                                    944 non-null    object        
 5   total_cases                                 944 non-null    float64       
 6   new_cases                                   944 non-null    float64       
 7   new_cases_smoothed                          944 non-null    float64       
 8   total_deaths                                944 non-null    float64       
 9   new_deaths

In [42]:
df['year'] = df['date'].dt.year
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month

In [43]:
df

,date,index,iso_code,continent,location,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,year,day,month
0,2020-01-04,183133,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,4,1
1,2020-01-05,183134,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,5,1
2,2020-01-06,183135,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,6,1
3,2020-01-07,183136,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,7,1
4,2020-01-08,183137,THA,Asia,Thailand,0.0,0.0,0.000,0.0,0.0,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,2022-07-31,184072,THA,Asia,Thailand,4592284.0,2108.0,2098.714,31404.0,19.0,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,31,7
940,2022-08-01,184073,THA,Asia,Thailand,4594127.0,1843.0,2362.000,31431.0,27.0,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,1,8
941,2022-08-02,184074,THA,Asia,Thailand,4596559.0,2432.0,2055.857,31463.0,32.0,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,2,8
942,2022-08-03,184075,THA,Asia,Thailand,4598725.0,2166.0,2093.571,31492.0,29.0,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,3,8


In [44]:
df = df.drop(columns=['date', 'location'])

In [45]:
df

,index,iso_code,continent,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,year,day,month
0,183133,THA,Asia,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,4,1
1,183134,THA,Asia,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,5,1
2,183135,THA,Asia,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,6,1
3,183136,THA,Asia,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,7,1
4,183137,THA,Asia,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,184072,THA,Asia,4592284.0,2108.0,2098.714,31404.0,19.0,30.714,64137.057,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,31,7
940,184073,THA,Asia,4594127.0,1843.0,2362.000,31431.0,27.0,34.571,64162.796,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,1,8
941,184074,THA,Asia,4596559.0,2432.0,2055.857,31463.0,32.0,29.286,64196.762,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,2,8
942,184075,THA,Asia,4598725.0,2166.0,2093.571,31492.0,29.0,28.857,64227.013,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,3,8


In [46]:
df['iso_code'] = df['iso_code'].apply(lambda x : 3166)

In [47]:
df['continent'] = df['continent'].apply(lambda x : 142)

In [48]:
df

,index,iso_code,continent,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,year,day,month
0,183133,3166,142,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,4,1
1,183134,3166,142,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,5,1
2,183135,3166,142,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,6,1
3,183136,3166,142,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,7,1
4,183137,3166,142,0.0,0.0,0.000,0.0,0.0,0.000,0.000,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2020,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,184072,3166,142,4592284.0,2108.0,2098.714,31404.0,19.0,30.714,64137.057,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,31,7
940,184073,3166,142,4594127.0,1843.0,2362.000,31431.0,27.0,34.571,64162.796,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,1,8
941,184074,3166,142,4596559.0,2432.0,2055.857,31463.0,32.0,29.286,64196.762,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,2,8
942,184075,3166,142,4598725.0,2166.0,2093.571,31492.0,29.0,28.857,64227.013,...,7.04,1.9,38.8,90.67,2.1,77.15,0.777,2022,3,8


In [51]:
df['total_vaccinations'].iloc[0:409]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     0.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     0.0
28     0.0
29     0.0
30     0.0
31     0.0
32     0.0
33     0.0
34     0.0
35     0.0
36     0.0
37     0.0
38     0.0
39     0.0
40     0.0
41     0.0
42     0.0
43     0.0
44     0.0
45     0.0
46     0.0
47     0.0
48     0.0
49     0.0
50     0.0
51     0.0
52     0.0
53     0.0
54     0.0
55     0.0
56     0.0
57     0.0
58     0.0
59     0.0
60     0.0
61     0.0
62     0.0
63     0.0
64     0.0
65     0.0
66     0.0
67     0.0
68     0.0
69     0.0
70     0.0
71     0.0
72     0.0
73     0.0
74     0.0
75     0.0
76     0.0
77     0.0
78     0.0
79     0.0
80     0.0
81     0.0
82     0.0
83     0.0
84     0.0
85     0.0
86     0.0
87     0.0
88     0.0
89     0.0
90     0.0

##### 